In [35]:
import sys
sys.path.append('D:\github\python')
import pandas as pd
import xlsxwriter
import re
import time
import numpy as np
import pandas as pd
# from Modul.modulku import praproses as pps
from Cek_typo import cek_typo as ct
from Normalisasi_KBBI import normalisasi_kbbi as nkbi
from modulku import praproses as pps
from modulku import StemNstopW as stm
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
from essay_scoring.ES import scoring as sc
import json

In [2]:
ct.norm_typo("dpan")

'depan'

In [84]:
data = pd.read_excel("data/data_train_A.xlsx")
res_id = data['RES_ID'].tolist()
respon = data['RESPONSE'].tolist()
print(len(res_id))
label = data['LABEL'].tolist()

# data = pd.read_excel("data/data_dev_A.xlsx")
# res_id = res_id+data['RES_ID'].tolist()
# respon = respon+data['RESPONSE'].tolist()
respon

268


['intetraksi/beradaptasi terhadap lingkungan yang baru.',
 'seperti jatuhnya meteor tsunami gempa bumi',
 'hanya tuhan yang tahu tantangan nya itu apaan',
 'mereka akan sulit beradaptasi',
 'Tempat tinggal, ekonomi, dan pekerjaan',
 'akan terganggu dengan kesehatanya karena merekanperlu beradaptasi dengan lingkungan yang baru.',
 'Mereka tidak akan di hormati oleh orang layaknya mereka di hormati di dalam tempat tinggal mereka yang lama',
 'mereka akan kehilangan lahan pertanian.',
 'Pada kasus-kasus seperti salah satunya yang terjadi di Travina, warga mungkin dipaksa untuk menjadi "pengungsi iklimâ€\x9d. Pengungsi iklim adalah orang-orang yang terpaksa meninggalkan komunitas atau negaranya karena bencana lingkungan.',
 'mendapat musibah dan melawannya',
 'karena mereka butuh tempat tinggal baru',
 'Akan kehilangan tempat tinggal yang lama atau komunitas karena bencana alam.',
 'mereka akan pindah ke tempat yang dulu',
 'beradaptasi',
 'Pengungsi iklim ini orang-orang yang terpaksa men

In [13]:
len(res_id)

483

In [14]:
len(res_id)

483

In [85]:
respon_list = list()
ixx = 1
oo = 0
for teks in respon:
    a = tknzr.tokenize(teks)
    a = " ".join(a)
#     a = teks
    a = pps.preprocessing(teks)
    a = pps.removePunc(a)
    a = ct.norm_typo(a)
    a = nkbi.norm_kbbi(a)
#     a = stm.stemmer_kata(a)
#     a = stm.stop_word(a)
    a = re.sub(' +', ' ',a)
    a = a.lstrip()
    respon_list.append(a)
    if ixx%10==0:
        #oo+=1
        print(".", end ="")
    if ixx%50==0:
        print(ixx, end="")
    ixx+=1

.....50.....100.....150.....200.....250.

In [101]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from joblib import dump
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(respon_list)
save_ = vectorizer.fit(respon)
dump(save_, 'model/tfdf_A.joblib')

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
new_labels =  kmeans.labels_
y = new_labels
len(y)
# respon_list

268

In [102]:
bobot_dokumen = list()
for i in X.toarray():
    bobot_dokumen.append((sum(i)))

In [113]:
f = sum(save_.transform(['tempat tinggal ekonomi dan pekerjaan']).toarray()[0])
idx = range(len(bobot_dokumen))
# bobot_dokumen.index(f)
jarak = list()
for i, ix in zip(bobot_dokumen, idx):
    jarak.append(abs(i-f))
print(jarak.index(min(jarak)))
respon[jarak.index(min(jarak))]

181


'kehilangan lingkungan kita akibat bencana .'

In [99]:
respon_list

['interaksi beradaptasi terhadap lingkungan yang baru',
 'seperti jatuhnya meteor tsunami gempa bumi',
 'hanya tuhan yang tahu tantangan nya itu apa',
 'mereka akan sulit beradaptasi',
 'tempat tinggal ekonomi dan pekerjaan',
 'akan terganggu dengan kesehatan karena mereka perlu beradaptasi dengan lingkungan yang baru',
 'mereka tidak akan di hormati oleh orang layaknya mereka di hormati di dalam tempat tinggal mereka yang lama',
 'mereka akan kehilangan lahan pertanian',
 'pada kasus kasus seperti salah satunya yang terjadi di travina warga mungkin dipaksa untuk menjadi pengungsi iklim pengungsi iklim adalah orang orang yang terpaksa meninggalkan komunitas atau negaranya karena bencana lingkungan',
 'mendapat musibah dan melawannya',
 'karena mereka butuh tempat tinggal baru',
 'akan kehilangan tempat tinggal yang lama atau komunitas karena bencana alam',
 'mereka akan pindah ke tempat yang dulu',
 'beradaptasi',
 'pengungsi iklim ini orang orang yang terpaksa meninggalkan komunitas a

In [79]:
min(jarak)

0.00402225104752274

In [20]:
dit_ ={
    'res_id':res_id[:268],
    'label':label[:268],
    'cluster_labe':y[:268],
    'respon_old':respon[:268],
    'respon':respon_list[:268]
}
h = pd.DataFrame.from_dict(dit_)
# h.to_excel('data/Data_A_Bersih_268.xlsx')
h.to_excel('data/data_Train_A_stemer(TRUE).xlsx') #data_dev_A
h.head()

,res_id,label,cluster_labe,respon_old,respon
0,TRA1,1,1,intetraksi/beradaptasi terhadap lingkungan yan...,interaksi adaptasi hadap lingkung yang baru
1,TRA2,0,1,seperti jatuhnya meteor tsunami gempa bumi,seperti jatuh meteor tsunami gempa bumi
2,TRA3,0,1,hanya tuhan yang tahu tantangan nya itu apaan,hanya tuhan yang tahu tantang nya itu apa
3,TRA4,1,1,mereka akan sulit beradaptasi,mereka akan sulit adaptasi
4,TRA5,1,1,"Tempat tinggal, ekonomi, dan pekerjaan",tempat tinggal ekonomi dan kerja
